In [1]:
import re
import nltk
from bs4 import BeautifulSoup

from urllib.request import urlopen

## Initial Cleaning

In [2]:
def html_tag_cleaner(text):
        clean_text = BeautifulSoup(text)
        return clean_text.get_text()

##  Bangla Word tokenization and stemming

In [3]:
def stem(word):
    regexp = r'^(.*?)(ের|েই|দের|ছিলেন|ির|ীয়|েও|টি|ও|তে|কে|গুলি|গুলো|গুলা|ছিলেন|ছিল|লেন|কের)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem

def tokenize_and_stem(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = []
    for t in tokens:
        if t.endswith('।'):
            t = t[:-len('।')]
        filtered_tokens.append(t.lower())
    stems = [stem(t) if len(t)>4 else t for t in tokens]
    
    return stems
    

<br/>
<br/><br/><br/>
# Getting data




In [4]:
import pandas as pd


posts = pd.read_csv('article_posts.csv', index_col=0)

In [5]:
posts.head()

,id,post_by_id,type,status
0,309048,147,article,এতদিন টাকা দিয়ে কল করেছেন এখন কল রিসিভ করে টাক...
1,309074,30,article,"কিছুদিন ধরেই ভাবছিলাম, বই পড়ার জন্য একটা ই-বুক..."
2,309075,31543,article,"<img style=""width: 120px; height: 152px;"" clas..."
3,309093,35355,article,"<p style=""margin:0in;margin-bottom:.0001pt;bac..."
4,309101,31585,article,&nbsp;ফজিলাতুন্নেছাকে না পাওয়ার ব্যাথায় বেদনা ...


<br>
## Clean unnecessary tags

In [6]:
posts['cleaned_status'] = posts['status'].apply(html_tag_cleaner)

/home/meftaul/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/meftaul/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.priyo.com/2014/03/30/61375.html'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' %

In [7]:
posts.tail()

,id,post_by_id,type,status,cleaned_status
1995,410791,41642,article,<br> » যারা নিয়মিত সকালের নাশতায় ডিম<br> খাও...,» যারা নিয়মিত সকালের নাশতায় ডিম খাওয়ার অভ্...
1996,410874,29373,article,ভাববেন না এখানে সবাই আপনার মত একই উদ্দেশ্য আসা...,ভাববেন না এখানে সবাই আপনার মত একই উদ্দেশ্য আসা...
1997,411096,2232,article,"<p style=""margin: 0px 0px 10px; color: rgb(51,...",হংকংয়ের চীনা তরুণ চাও হোই-লিয়াং মা-বাবার প্রিয়...
1998,411097,2232,article,"<p style=""margin: 0px 0px 10px; color: rgb(51,...",পরিবারের লোকজন ভেবেছিলেন কোন কালা জাদু ভর করেছ...
1999,411099,2232,article,"<span style=""color: rgb(51, 51, 51); font-fami...",পবিত্র হজ মহান আল্লাহর এক বড় হুকুম। এটি ইসলামে...


<br>
## Create tokens & frequency distribution

In [8]:
posts['tokens'] = [tokenize_and_stem(t) for t in posts.cleaned_status]
posts['frequency_dist'] = posts['tokens'].apply(nltk.FreqDist)

In [9]:
posts.tail()

,id,post_by_id,type,status,cleaned_status,tokens,frequency_dist
1995,410791,41642,article,<br> » যারা নিয়মিত সকালের নাশতায় ডিম<br> খাও...,» যারা নিয়মিত সকালের নাশতায় ডিম খাওয়ার অভ্...,"[», যারা, নিয়মিত, সকাল, নাশতায়, ডিম, খাওয়ার...","{'পরিস্কার': 1, 'খেয়ে': 1, 'ডিম': 1, '»': 6, ..."
1996,410874,29373,article,ভাববেন না এখানে সবাই আপনার মত একই উদ্দেশ্য আসা...,ভাববেন না এখানে সবাই আপনার মত একই উদ্দেশ্য আসা...,"[ভাববেন, না, এখানে, সবাই, আপনার, মত, একই, উদ্দ...","{'করছি': 1, 'সামনে': 1, 'হয়': 2, 'হলে': 1, 'ব..."
1997,411096,2232,article,"<p style=""margin: 0px 0px 10px; color: rgb(51,...",হংকংয়ের চীনা তরুণ চাও হোই-লিয়াং মা-বাবার প্রিয়...,"[হংকংয়, চীনা, তরুণ, চাও, হোই-লিয়াং, মা-বাবার, ...","{'বন্ধুরা': 1, 'খুঁজতে।': 1, 'গড়ে': 1, 'এটাই':..."
1998,411097,2232,article,"<p style=""margin: 0px 0px 10px; color: rgb(51,...",পরিবারের লোকজন ভেবেছিলেন কোন কালা জাদু ভর করেছ...,"[পরিবার, লোকজন, ভেবে, কোন, কালা, জাদু, ভর, করে...","{'থাকেন': 1, 'ঝুলে': 1, 'রাজীব': 1, 'নিশ্বাসে'..."
1999,411099,2232,article,"<span style=""color: rgb(51, 51, 51); font-fami...",পবিত্র হজ মহান আল্লাহর এক বড় হুকুম। এটি ইসলামে...,"[পবিত্র, হজ, মহান, আল্লাহর, এক, বড়, হুকুম।, এট...","{'প্রবেশ': 1, 'হাজীর': 1, 'তাকে': 4, 'চলাকালীন..."


In [10]:
posts['type'].value_counts()

article    2000
Name: type, dtype: int64

<br>
<br>
## Build Vocabulary

In [11]:
beshto_vocabulary = [word for tokens in posts['tokens'] for word in tokens]
#for tokens in posts['tokens'][:10]:
#    for word in tokens:
#        beshto_vocabulary.append(word)

In [12]:
len(beshto_vocabulary)

490160

In [13]:
beshto_vocabulary = nltk.FreqDist(beshto_vocabulary)
print(type(beshto_vocabulary))

<class 'nltk.probability.FreqDist'>


In [15]:
beshto_vocabulary.most_common(50)

[(',', 18894),
 ('!', 4594),
 ('করে', 4571),
 ('না', 4050),
 (')', 3284),
 ('?', 3125),
 ('আর', 2939),
 ('...', 2838),
 ('এই', 2806),
 ('(', 2705),
 ('আমার', 2547),
 ('আমি', 2286),
 ('থেকে', 2196),
 ('ও', 2136),
 ('তার', 1973),
 (':', 1890),
 ('যে', 1863),
 ('কি', 1857),
 ('তা', 1807),
 ('এবং', 1781),
 ('করতে', 1746),
 ('জন্য', 1714),
 ('কিন্তু', 1683),
 ('আমা', 1664),
 ('--', 1624),
 ('কিছু', 1615),
 ('অনেক', 1501),
 ('দিয়ে', 1447),
 ('.', 1425),
 ('মানুষ', 1393),
 ('সাথে', 1376),
 ('হয়ে', 1330),
 ('এক', 1329),
 ('কোন', 1308),
 ('বা', 1299),
 ('হয়', 1286),
 ('করা', 1285),
 ('এর', 1284),
 ('।', 1268),
 ('হবে', 1260),
 ('একটি', 1257),
 ('কথা', 1253),
 ('একটা', 1230),
 ('না।', 1220),
 ('নিয়ে', 1220),
 ('এ', 1201),
 ('মনে', 1148),
 ('আমরা', 1145),
 ('সে', 1128),
 ('সেই', 1016)]

<br>
<br>
### Words to feature

In [16]:
vocabulary_list = list(beshto_vocabulary.keys())
len(vocabulary_list)

52553

In [17]:
def find_features(token_list):
    features = {}
    for w in vocabulary_list:
        features[w] = (w in token_list)
    return features

In [ ]:
posts['feature'] = posts['tokens'].apply(find_features)